# Spotify Recommendation - Finding your music taste!

## Import necessary libraries

In [1]:
import pandas as pd
from requests_oauthlib import OAuth2Session
from oauthlib.oauth2 import BackendApplicationClient
import requests
import time

## Using Credentials to access OAuth for spotify

In [2]:
# Your credentials
client_id = '0af53b02c90245839d0f6d6f3bf45fb1'
client_secret = '89783cd95eac43eebbfc7f2e2a885dbb'

# Create a session
client = BackendApplicationClient(client_id=client_id)
oauth = OAuth2Session(client=client)

# Get token for the session
token = oauth.fetch_token(
    token_url='https://accounts.spotify.com/api/token',
    client_id=client_id,
    client_secret=client_secret,
    include_client_id=True
)

## Ideas for model

Before we start, we need to consider a couple of things when recommending songs:

- Is the track available for specified country?

- Did the user dislike similar tracks or this specified track?

Features for the model:

- Maybe consider getting input from users of language/country of song you want?

## Initializing Token and Getting API Request

In [3]:
# The headers to provide the access token for authentication
headers = {
    'Authorization': f'Bearer {token["access_token"]}',  # OAuth 2.0 Bearer token for authorization
    'Content-Type': 'application/json'  # Indicates the media type of the resource
}

In [5]:
token_expiry = time.time() + token['expires_in'] # gives time of expiry in total seconds
token_expiry_time = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(token_expiry)) # converting to readable time format

'2024-03-11 23:55:37'

we need to check if time.time() is going to exceed the token_expiry_date. If so then renew the token!

In [8]:
# Function to refresh the token
def refresh_token_if_needed(client_id, client_secret, token, token_expiry):
    # Check if the current time is within 60 seconds of expiry or past it
    if time.time() > token_expiry - 60:
        client = BackendApplicationClient(client_id=client_id)
        oauth = OAuth2Session(client=client)
        token = oauth.fetch_token(
            token_url='https://accounts.spotify.com/api/token',
            client_id=client_id,
            client_secret=client_secret,
            include_client_id=True
        )
        # Calculate new expiry time
        token_expiry = time.time() + token['expires_in']
        
    return token, token_expiry

# Usage before making an API request
# token, token_expiry = refresh_token_if_needed(client_id, client_secret, token, token_expiry)


## Recommending songs based on user chosen track

In [6]:
search_query = 'track:Mr. Brightside artist:The Killers'
search_type = 'track'
headers = {
    'Authorization': f'Bearer {token["access_token"]}'
}

response = requests.get(f'https://api.spotify.com/v1/search?q={search_query}&type={search_type}', headers=headers)

if response.status_code == 200:
    search_results = response.json()
    # You can iterate over search_results to find your specific track
    print(search_results)
else:
    print('Failed to fetch search results:', response.json())


{'tracks': {'href': 'https://api.spotify.com/v1/search?query=track%3AMr.+Brightside+artist%3AThe+Killers&type=track&offset=0&limit=20', 'items': [{'album': {'album_type': 'album', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/0C0XlULifJtAgn6ZNCW2eu'}, 'href': 'https://api.spotify.com/v1/artists/0C0XlULifJtAgn6ZNCW2eu', 'id': '0C0XlULifJtAgn6ZNCW2eu', 'name': 'The Killers', 'type': 'artist', 'uri': 'spotify:artist:0C0XlULifJtAgn6ZNCW2eu'}], 'available_markets': ['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA', 'CL', 'CO', 'CR', 'CY', 'CZ', 'DK', 'DO', 'DE', 'EC', 'EE', 'SV', 'FI', 'FR', 'GR', 'GT', 'HN', 'HK', 'HU', 'IS', 'IE', 'IT', 'LV', 'LT', 'LU', 'MY', 'MT', 'MX', 'NL', 'NZ', 'NI', 'NO', 'PA', 'PY', 'PE', 'PH', 'PL', 'PT', 'SG', 'SK', 'ES', 'SE', 'CH', 'TW', 'TR', 'UY', 'US', 'GB', 'AD', 'LI', 'MC', 'ID', 'JP', 'TH', 'VN', 'RO', 'IL', 'ZA', 'SA', 'AE', 'BH', 'QA', 'OM', 'KW', 'EG', 'MA', 'DZ', 'TN', 'LB', 'JO', 'PS', 'IN', 'BY', 'KZ', 'MD', 'UA', 'AL', '

In [7]:
# Example of getting track details from Spotify
track_id = '3n3Ppam7vgaVa1iaRUc9Lp'  # Example track ID
response = oauth.get(f'https://api.spotify.com/v1/tracks/{track_id}', headers=headers)
track_details = response.json()

# Check for successful access and print the track details
if response.status_code == 200:
    print(track_details)
else:
    print('Failed to fetch track details:',track_details)

{'album': {'album_type': 'album', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/0C0XlULifJtAgn6ZNCW2eu'}, 'href': 'https://api.spotify.com/v1/artists/0C0XlULifJtAgn6ZNCW2eu', 'id': '0C0XlULifJtAgn6ZNCW2eu', 'name': 'The Killers', 'type': 'artist', 'uri': 'spotify:artist:0C0XlULifJtAgn6ZNCW2eu'}], 'available_markets': ['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CL', 'CO', 'CR', 'CY', 'CZ', 'DK', 'DO', 'DE', 'EC', 'EE', 'SV', 'FI', 'FR', 'GR', 'GT', 'HN', 'HK', 'HU', 'IS', 'IE', 'IT', 'LV', 'LT', 'LU', 'MY', 'MT', 'MX', 'NL', 'NZ', 'NI', 'NO', 'PA', 'PY', 'PE', 'PH', 'PL', 'PT', 'SG', 'SK', 'ES', 'SE', 'CH', 'TW', 'TR', 'UY', 'GB', 'AD', 'LI', 'MC', 'ID', 'JP', 'TH', 'VN', 'RO', 'IL', 'ZA', 'SA', 'AE', 'BH', 'QA', 'OM', 'KW', 'EG', 'MA', 'DZ', 'TN', 'LB', 'JO', 'PS', 'IN', 'BY', 'KZ', 'MD', 'UA', 'AL', 'BA', 'HR', 'ME', 'MK', 'RS', 'SI', 'KR', 'BD', 'PK', 'LK', 'GH', 'KE', 'NG', 'TZ', 'UG', 'AG', 'AM', 'BS', 'BB', 'BZ', 'BT', 'BW', 'BF', 'CV', 'CW', 'DM', 'FJ